In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from scipy import signal

In [2]:
# ------------------------------
# A1. Load and Preprocess Dataset
# ------------------------------
data_dir = "drownsiness_dataset"  # Update with correct path
img_size = (64, 64)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 38400 images belonging to 2 classes.
Found 9600 images belonging to 2 classes.


In [3]:
model = keras.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, validation_data=val_generator, epochs=10)


C:\Users\venka\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\venka\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1200/1200 ━━━━━━━━━━━━━━━━━━━━ 170s 139ms/step - accuracy: 0.8828 - loss: 0.2655 - val_accuracy: 0.9379 - val_loss: 0.1866
Epoch 2/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 182s 152ms/step - accuracy: 0.9712 - loss: 0.0846 - val_accuracy: 0.9356 - val_loss: 0.1728
Epoch 3/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 164s 136ms/step - accuracy: 0.9796 - loss: 0.0582 - val_accuracy: 0.9379 - val_loss: 0.1900
Epoch 4/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 165s 137ms/step - accuracy: 0.9852 - loss: 0.0447 - val_accuracy: 0.9203 - val_loss: 0.2284
Epoch 5/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 168s 140ms/step - accuracy: 0.9886 - loss: 0.0343 - val_accuracy: 0.9366 - val_loss: 0.1957
Epoch 6/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 160s 133ms/step - accuracy: 0.9918 - loss: 0.0274 - val_accuracy: 0.9025 - val_loss: 0.2953
Epoch 7/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 155s 129ms/step - accuracy: 0.9925 - loss: 0.0227 - val_accuracy: 0.8969 - val_loss: 0.3546
Epoch 8/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 156s 129ms/step - accuracy: 0.9

In [ ]:

# ------------------------------
# A1 (Lab 05). Visualizing Feature Maps
# ------------------------------
import numpy as np
import matplotlib.pyplot as plt

# Ensure the model is called at least once to define inputs
dummy_input = np.random.rand(1, 64, 64, 3)  # Adjust shape as needed
_ = model.predict(dummy_input)  # Forward pass to initialize layers

# Extract the outputs of all convolutional layers
layer_outputs = [layer.output for layer in model.layers if isinstance(layer, layers.Conv2D)]

# Create the activation model
activation_model = keras.models.Model(inputs=model.input, outputs=layer_outputs)
# Select a sample image from the training generator and preprocess it
sample_img, _ = next(train_generator)
sample_img = np.expand_dims(sample_img[0], axis=0)  # Add batch dimension

# **Ensure model is explicitly called**
_ = model(sample_img)  # This initializes the layers properly

# Extract the outputs of all convolutional layers
layer_outputs = [layer.output for layer in model.layers if isinstance(layer, layers.Conv2D)]

# Create the activation model
activation_model = keras.models.Model(inputs=model.input, outputs=layer_outputs)

# Get feature maps
activations = activation_model.predict(sample_img)

# Plot feature maps for each convolutional layer
for layer_index, activation in enumerate(activations):
    num_filters = activation.shape[-1]  # Number of filters in the layer
    fig, axes = plt.subplots(1, min(num_filters, 5), figsize=(15, 5))

    for i in range(min(num_filters, 5)):  # Show up to 5 filters
        ax = axes[i] if num_filters > 1 else axes
        ax.imshow(activation[0, :, :, i], cmap='gray')
        ax.set_title(f'Feature Map {i+1}')
        ax.axis('off')

    plt.show()



In [8]:
# ------------------------------
# A7 (Lab 05). Implementing Custom CNN for Project
# ------------------------------
project_model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

project_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_project = project_model.fit(train_generator, validation_data=val_generator, epochs=15)

loss_project, accuracy_project = project_model.evaluate(val_generator)
print(f"Project Model Validation Accuracy: {accuracy_project * 100:.2f}%")


Epoch 1/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 136s 110ms/step - accuracy: 0.8700 - loss: 0.2831 - val_accuracy: 0.9236 - val_loss: 0.1807
Epoch 2/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 130s 108ms/step - accuracy: 0.9771 - loss: 0.0649 - val_accuracy: 0.9008 - val_loss: 0.2413
Epoch 3/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 142s 108ms/step - accuracy: 0.9872 - loss: 0.0396 - val_accuracy: 0.9256 - val_loss: 0.2124
Epoch 4/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 133s 110ms/step - accuracy: 0.9906 - loss: 0.0313 - val_accuracy: 0.9314 - val_loss: 0.1902
Epoch 5/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 132s 110ms/step - accuracy: 0.9905 - loss: 0.0309 - val_accuracy: 0.9239 - val_loss: 0.2289
Epoch 6/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 131s 108ms/step - accuracy: 0.9928 - loss: 0.0242 - val_accuracy: 0.9490 - val_loss: 0.1478
Epoch 7/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 132s 110ms/step - accuracy: 0.9928 - loss: 0.0225 - val_accuracy: 0.9152 - val_loss: 0.2593
Epoch 8/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 132s 109ms/step - ac